In [3]:
import psycopg
from psycopg import sql
import os
import glob

In [4]:
HOST = 'localhost'
DBNAME = 'assignment_2'
USER = 'postgres'
PASSWORD = '080525'

In [5]:
CONNECTION = f"host={HOST} dbname={DBNAME} user={USER} password={PASSWORD}"

In [11]:
with psycopg.connect(CONNECTION) as conn:
    with conn.cursor() as cur:
        cur.execute("""
            create table ex_table3 (
                num integer NOT NULL,
                name character varying (45),
                "studentID" integer,
                address character varying (45),
                phone character varying (45),
                CONSTRAINT ex_table3_pkey PRIMARY KEY (num)
                )
                """)
        
        SQL_INSERT = """INSERT INTO ex_table3(num, name, "studentID", address, phone) values (%s, %s, %s, %s, %s)"""
        with conn.pipeline() as p:
            cur.execute(SQL_INSERT, (0, 'Hong', 1, 'seoul', '010-0000-0000'))
            cur.execute(SQL_INSERT, (1, 'Lee', 2, 'Incheon', '010-1111-1111'))
            cur.execute(SQL_INSERT, (2, 'Kim', 3, 'Suwon', '010-2222-2222'))
            cur.execute(SQL_INSERT, (3, 'Kwon', 4, 'Seoul', '010-1111-3333'))
            cur.execute(SQL_INSERT, (4, 'Choi', 5, 'Busan', '010-4444-1111'))
            
        cur.execute("select * from ex_table3")
        records = cur.fetchall()
        
        for record in records:
            print(record)

(0, 'Hong', 1, 'seoul', '010-0000-0000')
(1, 'Lee', 2, 'Incheon', '010-1111-1111')
(2, 'Kim', 3, 'Suwon', '010-2222-2222')
(3, 'Kwon', 4, 'Seoul', '010-1111-3333')
(4, 'Choi', 5, 'Busan', '010-4444-1111')


In [9]:
with psycopg.connect(CONNECTION) as conn:
    
    conn.execute ("""
        create table if not exists "department"
        ("dept_name" varchar(20),
         "building" varchar(15),
         "budget" numeric(12,2), check (budget > 0),
         primary key ("dept_name")
        )
        """)
    
    conn.execute ("""
        create table if not exists "course"
        ("course_id" varchar(8),
         "title" varchar(50),
         "dept_name" varchar(20),
         "credits" numeric(2,0) check ("credits" > 0),
         primary key ("course_id"),
         foreign key ("dept_name") references "department" ("dept_name")
         on delete set null
        )
        """)
    
    conn.commit()

In [17]:
with psycopg.connect(CONNECTION) as conn:
    
    conn.execute(
    sql.SQL("""
    create table if not exists {table}(
    {column1} varchar(5),
    {column2} varchar(20) not null,
    {column3} varchar(20),
    {column4} numeric(8,2) check ({column4} > 29000),
    primary key ({column1}),
    foreign key ({column3}) references {ref_table}({ref_column})
    on delete set null)""")
    .format (table = sql.Identifier('instructor'),
             column1 = sql.Identifier ('id'),
             column2 = sql.Identifier ('name'),
             column3 = sql.Identifier ('dept_name'),
             column4 = sql.Identifier ('salary'),
             ref_table = sql.Identifier ('department'),
             ref_column = sql.Identifier('dept_name')
            ))
    
    cur = conn.execute("""
        select "table_name"
        from "information_schema"."tables"
        where "table_schema" = %s
        order by "table_name";
        """,('public',))
    
    records = cur.fetchall()
    for record in records:
        print(record)
        
    conn.commit()

('course',)
('department',)
('ex_table2',)
('ex_table3',)
('instructor',)


In [22]:
with psycopg.connect(CONNECTION) as conn:
    
    conn.execute (sql.SQL("DELETE FROM {}").format(sql.Identifier('public', 'department')))
    
    with conn.pipeline() as p:
        query = sql.SQL ("INSERT INTO {} VALUES (%s, %s, %s)"). format(sql.Identifier('public', 'department'))
        
        conn.execute(query, ('Biology', 'Watson', '90000'))
        conn.execute(query, ('CS', 'Watson', '100000'))
        conn.execute(query, ('KOR', 'Taylor', '80000'))
        conn.execute(query, ('Elec', 'Painter', '120000'))
        conn.execute(query, ('Eng', 'Painter', '50000'))
        conn.execute(query, ('Music', 'Packard', '80000'))
        conn.execute(query, ('Finance', 'Watson', '130000'))
       
    cur = conn.execute("""select * from "department"; """)
    
    records = cur.fetchall()
    for record in records:
        print(record)
        
    conn.commit()

('Biology', 'Watson', Decimal('90000.00'))
('CS', 'Watson', Decimal('100000.00'))
('KOR', 'Taylor', Decimal('80000.00'))
('Elec', 'Painter', Decimal('120000.00'))
('Eng', 'Painter', Decimal('50000.00'))
('Music', 'Packard', Decimal('80000.00'))
('Finance', 'Watson', Decimal('130000.00'))


In [24]:
with psycopg.connect(CONNECTION) as conn:
    
    conn.execute ('DELETE FROM "course"')
    query = sql.SQL ("INSERT INTO {} VALUES ({})"). format(
            sql.Identifier('public', 'course'),
            sql.SQL(',').join(sql.Placeholder()*4)
    )
    
    
    with conn.pipeline() as p:
        
        
        conn.execute(query, ('BIO-101', 'Intro. to Biology','Biology','4'))
        conn.execute(query, ('BIO-301', 'Genetics','Biology','4'))
        conn.execute(query, ('BIO-399', 'Computational Biology','Biology','3'))
        conn.execute(query, ('CS-101', 'Database Sysyem','CS','3'))
        conn.execute(query, ('CS-201', 'Deep Learning','CS','4'))
        conn.execute(query, ('FIN-201', 'Investment Banking','Finance','4'))
        conn.execute(query, ('MU-199', 'Music Video Production' ,'Music','3'))
       
    cur = conn.execute("""select * from "department"; """)
    
    records = cur.fetchall()
    for record in records:
        print(record)
        
    conn.commit()

('Biology', 'Watson', Decimal('90000.00'))
('CS', 'Watson', Decimal('100000.00'))
('KOR', 'Taylor', Decimal('80000.00'))
('Elec', 'Painter', Decimal('120000.00'))
('Eng', 'Painter', Decimal('50000.00'))
('Music', 'Packard', Decimal('80000.00'))
('Finance', 'Watson', Decimal('130000.00'))


In [37]:
with psycopg.connect(CONNECTION) as conn:
    
    conn.execute("""DELETE FROM "public"."instructor";""")
    SQL_INSERT_INSTRUCTOR = "INSERT INTO {0} ({1}) VALUES ({2})"
    LIST_INSTRUCTOR_INFO = [('10101', 'Kwon', 'CS', '65000'),
                            ('20202', 'Se', 'CS', '75000'),
                            ('10102', 'In', 'Biology', '90000'),
                            ('10103', 'Chpo', 'Music', '65000'),
                            ('20212', 'Lee', 'Finance', '40000'),
                            ('20221', 'Yeom', 'Finance', '70000'),
                            ('20222', 'Jo', 'Biology', '70000'),
                            ('10502', 'Seo', 'Biology', '65000'),
                            ('11132', 'Park', 'CS', '72000'),
                            ('58135', 'Kim', 'CS', '92000'),
                            ('16428', 'Lim', 'Eng', '80000')
                           ]
    COLNAMES_INSTRUCTOR = ['id', 'name', 'dept_name', 'salary']
    
    with conn.pipeline() as p:
        for data in LIST_INSTRUCTOR_INFO:
            query = sql.SQL(SQL_INSERT_INSTRUCTOR).format(
            sql.Identifier('public', 'instructor'),
            sql.SQL(', ').join(map(sql.Identifier, COLNAMES_INSTRUCTOR)),
            sql.SQL(', ').join(sql.Placeholder()*len(COLNAMES_INSTRUCTOR))
            )
            conn.execute(query, data)
            
    query = sql.SQL("SELECT * FROM {}").format(sql.Identifier('public','instructor'))
    
    records = conn.execute(query).fetchall()
    for record in records:
        print(record)
        
    conn.commit()

('10101', 'Kwon', 'CS', Decimal('65000.00'))
('20202', 'Se', 'CS', Decimal('75000.00'))
('10102', 'In', 'Biology', Decimal('90000.00'))
('10103', 'Chpo', 'Music', Decimal('65000.00'))
('20212', 'Lee', 'Finance', Decimal('40000.00'))
('20221', 'Yeom', 'Finance', Decimal('70000.00'))
('20222', 'Jo', 'Biology', Decimal('70000.00'))
('10502', 'Seo', 'Biology', Decimal('65000.00'))
('11132', 'Park', 'CS', Decimal('72000.00'))
('58135', 'Kim', 'CS', Decimal('92000.00'))
('16428', 'Lim', 'Eng', Decimal('80000.00'))
